In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
population_data = pd.read_csv('../data/Population_E_All_Data_NOFLAG.csv', encoding = "ISO-8859-1")

In [3]:
element_code_map = population_data[['Element Code', 'Element']].value_counts().reset_index(name='count')
element_code_map = element_code_map.sort_values(by=['Element Code'])[['Element Code', 'Element']].reset_index(drop=True)
area_code_map = population_data[['Area Code', 'Area']].value_counts().reset_index(name='count')
area_code_map = area_code_map.sort_values(by=['Area Code'])[['Area Code', 'Area']].reset_index(drop=True)

In [4]:
def get_list_of_countries():
    countries = area_code_map[area_code_map['Area Code'] < 1000]
    # Removing China because it's an aggregation of multiple entries.
    countries = countries[countries['Area'] != 'China'].reset_index(drop=True)
    return countries

In [5]:
countries = get_list_of_countries()

In [6]:
filtered_population_data = population_data[
    (population_data['Area Code'].isin(get_list_of_countries()['Area Code']))
    & (population_data['Element Code'] == 511)
].reset_index(drop=True)
del population_data

In [7]:
mega_dataset = filtered_population_data.drop(['Item Code', 'Item'], axis=1)

In [8]:
mega_dataset

,Area Code,Area,Element Code,Element,Unit,Y1950,Y1951,Y1952,Y1953,Y1954,...,Y2091,Y2092,Y2093,Y2094,Y2095,Y2096,Y2097,Y2098,Y2099,Y2100
0,2,Afghanistan,511,Total Population - Both sexes,1000 persons,7752.118,7840.156,7935.997,8039.694,8151.317,...,76578.450,76448.747,76304.424,76146.060,75974.256,75789.684,75593.009,75384.981,75166.352,74937.964
1,3,Albania,511,Total Population - Both sexes,1000 persons,1263.174,1287.500,1316.093,1348.112,1382.898,...,1329.788,1301.302,1273.239,1245.593,1218.363,1191.538,1165.112,1139.106,1113.510,1088.336
2,4,Algeria,511,Total Population - Both sexes,1000 persons,8872.247,9023.269,9186.138,9364.371,9560.149,...,70396.145,70476.915,70547.373,70607.089,70655.555,70692.182,70716.265,70726.991,70723.450,70704.619
3,5,American Samoa,511,Total Population - Both sexes,1000 persons,18.940,19.293,19.542,19.695,19.753,...,40.991,40.469,39.937,39.401,38.861,38.323,37.784,37.238,36.697,36.148
4,6,Andorra,511,Total Population - Both sexes,1000 persons,6.196,6.689,7.247,7.865,8.525,...,63.002,62.934,62.874,62.806,62.740,62.675,62.611,62.542,62.469,62.400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239,205,Western Sahara,511,Total Population - Both sexes,1000 persons,13.766,16.062,17.570,18.726,19.837,...,1327.083,1333.688,1340.234,1346.694,1353.069,1359.380,1365.602,1371.751,1377.814,1383.771
240,249,Yemen,511,Total Population - Both sexes,1000 persons,4661.282,4717.152,4776.179,4837.601,4900.848,...,54508.720,54394.333,54270.579,54137.799,53996.338,53846.537,53688.764,53523.395,53350.779,53171.323
241,248,Yugoslav SFR,511,Total Population - Both sexes,1000 persons,16366.126,16623.077,16873.949,17115.630,17346.245,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
242,251,Zambia,511,Total Population - Both sexes,1000 persons,2310.442,2368.604,2431.344,2498.465,2569.748,...,74381.597,75203.383,76019.211,76828.802,77631.916,78428.394,79218.087,80000.947,80776.966,81546.200


In [9]:
climate_data = pd.read_csv('../data/Environment_Temperature_change_E_All_Data_NOFLAG.csv', encoding = "ISO-8859-1")

In [10]:
filtered_climate_data = climate_data[
    (climate_data['Months Code'] == 7020)
    & (climate_data['Element Code'] == 7271)
    & (climate_data['Area Code'].isin(countries['Area Code']))
].reset_index(drop=True)
del climate_data

In [11]:
filtered_climate_data = filtered_climate_data.drop(['Months Code', 'Months'], axis=1)

In [12]:
mega_dataset = mega_dataset.append(
    filtered_climate_data
).sort_values(
    by=['Area Code', 'Element']
).reset_index(drop=True)

In [13]:
mega_dataset

,Area Code,Area,Element Code,Element,Unit,Y1950,Y1951,Y1952,Y1953,Y1954,...,Y2091,Y2092,Y2093,Y2094,Y2095,Y2096,Y2097,Y2098,Y2099,Y2100
0,1,Armenia,7271,Temperature change,°C,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Armenia,511,Total Population - Both sexes,1000 persons,NaN,NaN,NaN,NaN,NaN,...,2175.894,2160.763,2145.660,2130.563,2115.465,2100.332,2085.130,2069.810,2054.326,2038.611
2,2,Afghanistan,7271,Temperature change,°C,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2,Afghanistan,511,Total Population - Both sexes,1000 persons,7752.118,7840.156,7935.997,8039.694,8151.317,...,76578.450,76448.747,76304.424,76146.060,75974.256,75789.684,75593.009,75384.981,75166.352,74937.964
4,3,Albania,7271,Temperature change,°C,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
475,280,Sint Maarten (Dutch part),511,Total Population - Both sexes,1000 persons,NaN,NaN,NaN,NaN,NaN,...,66.516,66.802,67.103,67.394,67.692,67.997,68.297,68.599,68.913,69.217
476,281,Saint-Martin (French part),511,Total Population - Both sexes,1000 persons,NaN,NaN,NaN,NaN,NaN,...,59.343,59.474,59.590,59.703,59.809,59.906,59.988,60.063,60.138,60.187
477,282,Saint Barthélemy,511,Total Population - Both sexes,1000 persons,NaN,NaN,NaN,NaN,NaN,...,10.413,10.387,10.358,10.325,10.300,10.270,10.241,10.209,10.179,10.149
478,299,Palestine,7271,Temperature change,°C,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
food_security_data = pd.read_csv('../data/Food_Security_Data_E_All_Data_NOFLAG.csv', encoding="ISO-8859-1")

In [15]:
filtered_food_security_data = food_security_data[
    (food_security_data['Area Code'].isin(countries['Area Code']))
    & (food_security_data['Item Code'].isin([210041, 210011, 210401, 210071, 210091, 210081]))
]
filtered_conflict_data = food_security_data[
    (food_security_data['Area Code'].isin(countries['Area Code']))
    & (food_security_data['Item Code'] == 21032)
]
del food_security_data


In [16]:
cols_to_process = [x for x in filtered_food_security_data.columns if x.startswith('Y2') and len(x) > 5]
for col in cols_to_process:
    year = 'Y' + str(int(col[5:]) - 1)
    filtered_food_security_data.loc[:,year] = filtered_food_security_data.loc[:, col]
filtered_food_security_data = filtered_food_security_data.drop(cols_to_process, axis=1)

In [17]:
filtered_food_security_data['Element Code'] = filtered_food_security_data['Item Code']
filtered_food_security_data['Element'] = filtered_food_security_data['Item']
filtered_food_security_data = filtered_food_security_data.drop(['Item', 'Item Code'], axis=1)

In [18]:
filtered_conflict_data = filtered_conflict_data.drop(['Element Code', 'Element'] + cols_to_process, axis=1)
filtered_conflict_data = filtered_conflict_data.rename(columns={'Item Code': 'Element Code', 'Item': 'Element'})

In [19]:
mega_dataset = mega_dataset.append(filtered_food_security_data)

In [20]:
mega_dataset = mega_dataset.append(filtered_conflict_data)

In [21]:
# Removing countries that do not have the full data
mega_dataset = mega_dataset[
    (mega_dataset['Area Code'].isin(filtered_food_security_data['Area Code'].unique()))
    & (mega_dataset['Area Code'].isin(filtered_climate_data['Area Code'].unique()))
].sort_values(
    by=['Area Code', 'Element']
).reset_index(drop=True)

countries = countries[countries['Area Code'].isin(mega_dataset['Area Code'].unique())]

In [22]:
# Removing projected and future data
unnecessary_cols = [x for x in mega_dataset.columns if x.startswith('Y2') and int(x[1:]) > 2020]
mega_dataset = mega_dataset.drop(unnecessary_cols, axis=1)

In [23]:
mega_dataset

,Area Code,Area,Element Code,Element,Unit,Y1950,Y1951,Y1952,Y1953,Y1954,...,Y2011,Y2012,Y2013,Y2014,Y2015,Y2016,Y2017,Y2018,Y2019,Y2020
0,1,Armenia,210081,Number of moderately or severely food insecure...,millions,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.5,0.4,0.4,NaN
1,1,Armenia,210011,Number of people undernourished (million) (3-y...,millions,NaN,NaN,NaN,NaN,NaN,...,0.1,0.1,<0.1,<0.1,<0.1,<0.1,<0.1,<0.1,0.1,NaN
2,1,Armenia,210071,Number of severely food insecure people (milli...,millions,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,<0.1,<0.1,<0.1,NaN
3,1,Armenia,21032,Political stability and absence of violence/te...,index,NaN,NaN,NaN,NaN,NaN,...,-0.06,0.15,0.11,-0.3,-0.25,-0.66,-0.62,-0.44,-0.51,NaN
4,1,Armenia,210091,Prevalence of moderate or severe food insecuri...,%,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,17.1,14.8,12.7,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1807,299,Palestine,210091,Prevalence of moderate or severe food insecuri...,%,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,26.3,26.3,26.3,NaN
1808,299,Palestine,210401,Prevalence of severe food insecurity in the to...,%,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,4.4,4.4,4.4,NaN
1809,299,Palestine,210041,Prevalence of undernourishment (percent) (3-ye...,%,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1810,299,Palestine,7271,Temperature change,°C,NaN,NaN,NaN,NaN,NaN,...,0.47,0.927,1.013,0.754,1.226,1.528,0.639,1.875,1.052,1.302


In [24]:
food_balance_data_old = pd.read_csv('../data/FoodBalanceSheetsHistoric_E_All_Data_NOFLAG.csv', encoding="ISO-8859-1")
food_balance_data_new = pd.read_csv('../data/FoodBalanceSheets_E_All_Data_NOFLAG.csv', encoding="ISO-8859-1")

In [25]:
# Taking the columns Food Suppy, Food Suppy Quantity, Losses respectively
filtered_food_balance_data_old = food_balance_data_old[
    (food_balance_data_old['Area Code'].isin(countries['Area Code']))
    & (food_balance_data_old['Element Code'].isin([664, 645, 5123]))
]

filtered_food_balance_data_new = food_balance_data_new[
    (food_balance_data_new['Area Code'].isin(countries['Area Code']))
    & (food_balance_data_new['Element Code'].isin([664, 645, 5123]))
]


In [26]:
filtered_food_balance_data_old = filtered_food_balance_data_old.drop(['Y2010', 'Y2011', 'Y2012', 'Y2013'], axis=1)

In [27]:
filtered_food_balance_data = pd.merge(
    left=filtered_food_balance_data_old,
    right=filtered_food_balance_data_new,
    how='outer',
    on=['Area Code', 'Element Code', 'Area', 'Item Code', 'Item', 'Element', 'Unit']
).reset_index(drop=True)

In [28]:
filtered_food_balance_data

,Area Code,Area,Item Code,Item,Element Code,Element,Unit,Y1961,Y1962,Y1963,...,Y2010,Y2011,Y2012,Y2013,Y2014,Y2015,Y2016,Y2017,Y2018,Y2019
0,2,Afghanistan,2901,Grand Total,664,Food supply (kcal/capita/day),kcal/capita/day,2999.00,2917.0,2698.00,...,2170.00,2152.00,2159.00,2196.00,2265.00,2250.00,2228.00,2303.00,2270.00,2273.00
1,2,Afghanistan,2903,Vegetal Products,664,Food supply (kcal/capita/day),kcal/capita/day,2752.00,2672.0,2438.00,...,1964.00,1953.00,1955.00,1993.00,2019.00,2038.00,2024.00,2108.00,2081.00,2087.00
2,2,Afghanistan,2941,Animal Products,664,Food supply (kcal/capita/day),kcal/capita/day,247.00,245.0,260.00,...,206.00,200.00,204.00,204.00,246.00,211.00,204.00,195.00,189.00,187.00
3,2,Afghanistan,2905,Cereals - Excluding Beer,5123,Losses,1000 tonnes,309.00,307.0,287.00,...,837.00,647.00,911.00,936.00,963.00,837.00,800.00,706.00,595.00,809.00
4,2,Afghanistan,2905,Cereals - Excluding Beer,645,Food supply quantity (kg/capita/yr),kg,309.04,300.2,269.18,...,193.16,188.94,181.12,183.98,181.97,182.03,181.61,185.87,181.87,181.56
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58363,181,Zimbabwe,2769,"Aquatic Animals, Others",664,Food supply (kcal/capita/day),kcal/capita/day,NaN,NaN,NaN,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
58364,181,Zimbabwe,2928,Miscellaneous,5123,Losses,1000 tonnes,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
58365,181,Zimbabwe,2680,Infant food,5123,Losses,1000 tonnes,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
58366,181,Zimbabwe,2899,Miscellaneous,5123,Losses,1000 tonnes,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
# Aggregating Food Suppy Quantity and Losses data and trimming Foody Supply data
food_balance_losses = filtered_food_balance_data[filtered_food_balance_data['Element Code'] == 5123].drop(['Item Code', 'Item'], axis=1)
food_balance_losses = food_balance_losses.groupby(by=['Area Code', 'Area', 'Element Code', 'Element', 'Unit']).sum().reset_index()
food_balance_losses.replace(0.0, np.nan, inplace=True)

food_balance_supply = filtered_food_balance_data[filtered_food_balance_data['Item Code'] == 2901].drop(['Item Code', 'Item'], axis=1)

food_balance_supply_qty = filtered_food_balance_data[filtered_food_balance_data['Element Code'] == 645].drop(['Item Code', 'Item'], axis=1)
food_balance_supply_qty = food_balance_supply_qty.groupby(by=['Area Code', 'Area', 'Element Code', 'Element', 'Unit']).sum().reset_index()
food_balance_supply_qty.replace(0.0, np.nan, inplace=True)

del food_balance_data_old
del food_balance_data_new




In [30]:
food_balance_supply

,Area Code,Area,Element Code,Element,Unit,Y1961,Y1962,Y1963,Y1964,Y1965,...,Y2010,Y2011,Y2012,Y2013,Y2014,Y2015,Y2016,Y2017,Y2018,Y2019
0,2,Afghanistan,664,Food supply (kcal/capita/day),kcal/capita/day,2999.0,2917.0,2698.0,2953.0,2956.0,...,2170.0,2152.0,2159.0,2196.0,2265.0,2250.0,2228.0,2303.0,2270.0,2273.0
176,3,Albania,664,Food supply (kcal/capita/day),kcal/capita/day,2223.0,2242.0,2156.0,2270.0,2254.0,...,3184.0,3237.0,3288.0,3293.0,3266.0,3280.0,3322.0,3326.0,3296.0,3338.0
408,4,Algeria,664,Food supply (kcal/capita/day),kcal/capita/day,1619.0,1569.0,1528.0,1540.0,1591.0,...,3250.0,3350.0,3366.0,3436.0,3424.0,3417.0,3443.0,3383.0,3382.0,3493.0
657,7,Angola,664,Food supply (kcal/capita/day),kcal/capita/day,1798.0,1819.0,1853.0,1862.0,1877.0,...,2358.0,2430.0,2389.0,2472.0,2517.0,2448.0,2408.0,2441.0,2451.0,2445.0
882,8,Antigua and Barbuda,664,Food supply (kcal/capita/day),kcal/capita/day,2008.0,2185.0,2052.0,2126.0,2222.0,...,2311.0,2386.0,2417.0,2408.0,2400.0,2405.0,2404.0,2446.0,2470.0,2485.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45713,250,Democratic Republic of the Congo,664,Food supply (kcal/capita/day),kcal/capita/day,NaN,NaN,NaN,NaN,NaN,...,1907.0,1920.0,1877.0,1891.0,1937.0,1933.0,1912.0,1947.0,1925.0,1913.0
50310,124,Libya,664,Food supply (kcal/capita/day),kcal/capita/day,NaN,NaN,NaN,NaN,NaN,...,3113.0,3057.0,3115.0,3114.0,3122.0,3125.0,3132.0,3116.0,3130.0,3141.0
53133,168,Papua New Guinea,664,Food supply (kcal/capita/day),kcal/capita/day,NaN,NaN,NaN,NaN,NaN,...,2136.0,2174.0,2203.0,2203.0,2225.0,2227.0,2222.0,2227.0,2243.0,2244.0
54956,196,Seychelles,664,Food supply (kcal/capita/day),kcal/capita/day,NaN,NaN,NaN,NaN,NaN,...,2933.0,2865.0,2901.0,2902.0,2957.0,2966.0,3004.0,3106.0,3112.0,3123.0


In [31]:
# Add the above aggregations to the mega dataset

mega_dataset = mega_dataset.append(food_balance_losses)
mega_dataset = mega_dataset.append(food_balance_supply)
mega_dataset = mega_dataset.append(food_balance_supply_qty).reset_index(drop=True)

In [32]:
mega_dataset

,Area Code,Area,Element Code,Element,Unit,Y1950,Y1951,Y1952,Y1953,Y1954,...,Y2011,Y2012,Y2013,Y2014,Y2015,Y2016,Y2017,Y2018,Y2019,Y2020
0,1,Armenia,210081,Number of moderately or severely food insecure...,millions,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.5,0.4,0.4,NaN
1,1,Armenia,210011,Number of people undernourished (million) (3-y...,millions,NaN,NaN,NaN,NaN,NaN,...,0.1,0.1,<0.1,<0.1,<0.1,<0.1,<0.1,<0.1,0.1,NaN
2,1,Armenia,210071,Number of severely food insecure people (milli...,millions,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,<0.1,<0.1,<0.1,NaN
3,1,Armenia,21032,Political stability and absence of violence/te...,index,NaN,NaN,NaN,NaN,NaN,...,-0.06,0.15,0.11,-0.3,-0.25,-0.66,-0.62,-0.44,-0.51,NaN
4,1,Armenia,210091,Prevalence of moderate or severe food insecuri...,%,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,17.1,14.8,12.7,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2347,255,Belgium,645,Food supply quantity (kg/capita/yr),kg,NaN,NaN,NaN,NaN,NaN,...,1493.47,1488.24,1542.01,1781.34,1667.62,1720.14,1651.36,1902.92,1895.15,NaN
2348,256,Luxembourg,645,Food supply quantity (kg/capita/yr),kg,NaN,NaN,NaN,NaN,NaN,...,1877.29,1915.48,1941.92,1857.42,1848.46,1896.22,1912.25,1906.44,1930.02,NaN
2349,272,Serbia,645,Food supply quantity (kg/capita/yr),kg,NaN,NaN,NaN,NaN,NaN,...,1436.63,1356.64,1459.05,1436.17,1469.79,1476.74,1454.39,1443.3,1462.18,NaN
2350,273,Montenegro,645,Food supply quantity (kg/capita/yr),kg,NaN,NaN,NaN,NaN,NaN,...,2132.72,2011.43,2085.65,2077.26,2169.48,2232.55,2246.37,2238.76,2250.12,NaN


In [33]:
# TODO: Add government expenditure data
expenditure_data = pd.read_csv('../data/Investment_GovernmentExpenditure_E_All_Data_NOFLAG.csv', encoding="ISO-8859-1")


In [34]:
filtered_expenditure_data = expenditure_data[
    (expenditure_data['Area Code'].isin(countries['Area Code']))
    & (expenditure_data['Element Code'] == 6111)
    & (expenditure_data['Item Code'] == 23161)
]
del expenditure_data

In [35]:
filtered_expenditure_data = filtered_expenditure_data.drop(labels=['Item Code', 'Item'], axis=1)


In [36]:
mega_dataset = mega_dataset.append(filtered_expenditure_data)

In [37]:
mega_dataset

,Area Code,Area,Element Code,Element,Unit,Y1950,Y1951,Y1952,Y1953,Y1954,...,Y2011,Y2012,Y2013,Y2014,Y2015,Y2016,Y2017,Y2018,Y2019,Y2020
0,1,Armenia,210081,Number of moderately or severely food insecure...,millions,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.5,0.4,0.4,NaN
1,1,Armenia,210011,Number of people undernourished (million) (3-y...,millions,NaN,NaN,NaN,NaN,NaN,...,0.1,0.1,<0.1,<0.1,<0.1,<0.1,<0.1,<0.1,0.1,NaN
2,1,Armenia,210071,Number of severely food insecure people (milli...,millions,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,<0.1,<0.1,<0.1,NaN
3,1,Armenia,21032,Political stability and absence of violence/te...,index,NaN,NaN,NaN,NaN,NaN,...,-0.06,0.15,0.11,-0.3,-0.25,-0.66,-0.62,-0.44,-0.51,NaN
4,1,Armenia,210091,Prevalence of moderate or severe food insecuri...,%,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,17.1,14.8,12.7,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6525,236,Venezuela (Bolivarian Republic of),6111,Share of Total Expenditure,%,NaN,NaN,NaN,NaN,NaN,...,3.45,2.04,1.16,4.16,NaN,NaN,NaN,NaN,NaN,NaN
6577,237,Viet Nam,6111,Share of Total Expenditure,%,NaN,NaN,NaN,NaN,NaN,...,2.64,2.32,3.69,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6665,249,Yemen,6111,Share of Total Expenditure,%,NaN,NaN,NaN,NaN,NaN,...,0.54,0.45,0.79,0.76,NaN,NaN,NaN,NaN,NaN,NaN
6732,251,Zambia,6111,Share of Total Expenditure,%,NaN,NaN,NaN,NaN,NaN,...,12.89,5.78,6.43,8.45,10.25,6.78,7.93,5.29,5.38,3.35


In [38]:
consumer_price_index_data = pd.read_csv('../data/ConsumerPriceIndices_E_All_Data_NOFLAG.csv', encoding="ISO-8859-1")

In [39]:
filtered_consumer_price_index_data = consumer_price_index_data[
    (consumer_price_index_data['Item Code'] == 23014)
    & (consumer_price_index_data['Area Code'].isin(countries['Area Code']))
    & (consumer_price_index_data['Months'] == 'January')
]


In [40]:
filtered_consumer_price_index_data = filtered_consumer_price_index_data.rename(columns={'Item': 'Element', 'Item Code': 'Element Code'})
filtered_consumer_price_index_data = filtered_consumer_price_index_data.sort_values('Area Code')
filtered_consumer_price_index_data.drop(['Months', 'Months Code'], axis=1, inplace=True)

In [41]:
filtered_consumer_price_index_data

,Area Code,Area,Element Code,Element,Unit,Y2000,Y2001,Y2002,Y2003,Y2004,...,Y2012,Y2013,Y2014,Y2015,Y2016,Y2017,Y2018,Y2019,Y2020,Y2021
348,1,Armenia,23014,Food price inflation,%,NaN,-5.093245,4.499744,2.559075,7.169667,...,5.995612,1.600868,3.941085,5.686249,-3.938860,1.803501,4.102667,0.487004,-1.743922,6.371823
24,2,Afghanistan,23014,Food price inflation,%,NaN,22.943765,11.612646,19.585062,8.592641,...,5.193009,5.066044,9.092442,2.957042,1.117175,4.698328,5.568586,-0.413473,5.985349,6.097938
96,3,Albania,23014,Food price inflation,%,NaN,7.093145,6.584784,5.926985,3.757915,...,1.444154,4.444146,2.650161,36.559391,-23.844752,8.480869,2.168390,4.346391,2.389434,1.178424
132,4,Algeria,23014,Food price inflation,%,NaN,-1.593977,4.138494,2.000131,4.502542,...,8.752048,10.470810,1.397886,5.637749,1.989390,6.855657,1.815704,1.348250,-1.184074,3.948059
168,6,Andorra,23014,Food price inflation,%,NaN,2.899967,2.231815,4.690659,2.703747,...,0.882818,2.773879,1.590489,1.905590,1.210428,2.207912,2.520252,1.053556,1.998262,1.618399
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5892,272,Serbia,23014,Food price inflation,%,NaN,86.345219,79.604262,8.595642,1.672241,...,3.550678,17.955112,-1.638478,0.483611,1.657754,1.209890,2.079002,2.953157,2.274975,0.435203
4368,273,Montenegro,23014,Food price inflation,%,NaN,2.217053,2.168966,2.122920,2.078789,...,3.048260,7.103375,-0.875843,1.786125,1.366027,1.226870,-0.517514,2.086028,2.653299,1.084081
6300,276,Sudan,23014,Food price inflation,%,NaN,NaN,NaN,NaN,NaN,...,23.066927,43.571133,25.774762,24.264132,9.913405,27.623577,56.211377,61.360641,67.735805,248.264690
6192,277,South Sudan,23014,Food price inflation,%,NaN,NaN,NaN,NaN,NaN,...,40.639862,33.315057,-6.731684,-6.758713,198.672353,369.533890,112.550566,67.133998,47.882087,41.953464


In [42]:
mega_dataset = mega_dataset.append(filtered_consumer_price_index_data).sort_values(by=['Area Code', 'Element Code']).reset_index(drop=True)

In [43]:
purchasing_power_data = pd.read_csv('../data/API_NY.GDP.PCAP.PP.CD_DS2_en_csv_v2_3469401.csv', encoding="ISO-8859-1")
purchasing_power_data.rename(columns={'ï»¿"Country Name"':'Country Name'}, inplace=True)
purchasing_power_data.drop(columns=['Unnamed: 65'], axis=1, inplace=True)
countries_purchasing_power = set(purchasing_power_data['Country Name'].unique())
countries_dataset = set(mega_dataset['Area'].unique())

In [44]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

purchasing_power_data['Country Name'] = purchasing_power_data["Country Name"].str.replace('Bahamas, The', 'Bahamas')
purchasing_power_data['Country Name'] = purchasing_power_data["Country Name"].str.replace('Bolivia', 'Bolivia (Plurinational State of)')
purchasing_power_data['Country Name'] = purchasing_power_data["Country Name"].str.replace(r'^Hong Kong SAR, China$', 'China, Hong Kong SAR')
purchasing_power_data['Country Name'] = purchasing_power_data["Country Name"].str.replace(r'^Macao SAR, China$', 'China, Macao SAR')
purchasing_power_data['Country Name'] = purchasing_power_data["Country Name"].str.replace(r'^China$', 'China, mainland')
purchasing_power_data['Country Name'] = purchasing_power_data["Country Name"].str.replace('Congo, Rep.', 'Congo')
purchasing_power_data['Country Name'] = purchasing_power_data["Country Name"].str.replace('Czech Republic', 'Czechia')
purchasing_power_data['Country Name'] = purchasing_power_data["Country Name"].str.replace("Cote d'Ivoire", "Côte d'Ivoire")
purchasing_power_data['Country Name'] = purchasing_power_data["Country Name"].str.replace("Korea, Dem. People's Rep.", "Democratic People's Republic of Korea")
purchasing_power_data['Country Name'] = purchasing_power_data["Country Name"].str.replace('Congo, Dem. Rep.', 'Democratic Republic of the Congo')
purchasing_power_data['Country Name'] = purchasing_power_data["Country Name"].str.replace('Egypt, Arab Rep.', 'Egypt')
purchasing_power_data['Country Name'] = purchasing_power_data["Country Name"].str.replace('Gambia, The', 'Gambia')
purchasing_power_data['Country Name'] = purchasing_power_data["Country Name"].str.replace('Iran, Islamic Rep.', 'Iran (Islamic Republic of)')
purchasing_power_data['Country Name'] = purchasing_power_data["Country Name"].str.replace('Kyrgyz Republic', 'Kyrgyzstan')
purchasing_power_data['Country Name'] = purchasing_power_data["Country Name"].str.replace('Lao PDR', "Lao People's Democratic Republic")
purchasing_power_data['Country Name'] = purchasing_power_data["Country Name"].str.replace('Micronesia, Fed. Sts.', 'Micronesia (Federated States of)')
purchasing_power_data['Country Name'] = purchasing_power_data["Country Name"].str.replace('Korea, Rep.', 'Republic of Korea')
purchasing_power_data['Country Name'] = purchasing_power_data["Country Name"].str.replace('Moldova', 'Republic of Moldova')
purchasing_power_data['Country Name'] = purchasing_power_data["Country Name"].str.replace('St. Kitts and Nevis', 'Saint Kitts and Nevis')
purchasing_power_data['Country Name'] = purchasing_power_data["Country Name"].str.replace('St. Lucia', 'Saint Lucia')
purchasing_power_data['Country Name'] = purchasing_power_data["Country Name"].str.replace('St. Vincent and the Grenadines', 'Saint Vincent and the Grenadines')
purchasing_power_data['Country Name'] = purchasing_power_data["Country Name"].str.replace('Slovak Republic', 'Slovakia')
purchasing_power_data['Country Name'] = purchasing_power_data["Country Name"].str.replace('United Kingdom', 'United Kingdom of Great Britain and Northern Ireland')
purchasing_power_data['Country Name'] = purchasing_power_data["Country Name"].str.replace('Tanzania', 'United Republic of Tanzania')
purchasing_power_data['Country Name'] = purchasing_power_data["Country Name"].str.replace('United States', 'United States of America')
purchasing_power_data['Country Name'] = purchasing_power_data["Country Name"].str.replace('Venezuela, RB', 'Venezuela (Bolivarian Republic of)')
purchasing_power_data['Country Name'] = purchasing_power_data["Country Name"].str.replace('Vietnam', 'Viet Nam')
purchasing_power_data['Country Name'] = purchasing_power_data["Country Name"].str.replace('Yemen, Rep.', 'Yemen')


In [45]:
purchasing_power_data = purchasing_power_data[purchasing_power_data['Country Name'].isin(countries_dataset)]
purchasing_power_data = purchasing_power_data.rename(columns={'Indicator Name': 'Element', 'Indicator Code': 'Element Code', 'Country Name': 'Area', 'Country Code': 'Area Code'})
area_areacode_dict = {v:k for k, v in dict(area_code_map.values).items()}


In [46]:
for i in purchasing_power_data.index:
    purchasing_power_data.loc[i, 'Area Code'] = area_areacode_dict[purchasing_power_data.loc[i, 'Area']]
purchasing_power_data.columns = purchasing_power_data.columns.str.replace(r'^19', 'Y19')
purchasing_power_data.columns = purchasing_power_data.columns.str.replace(r'^20', 'Y20')
purchasing_power_data['Unit'] = '$'

In [47]:
mega_dataset

,Area Code,Area,Element Code,Element,Unit,Y1950,Y1951,Y1952,Y1953,Y1954,...,Y2012,Y2013,Y2014,Y2015,Y2016,Y2017,Y2018,Y2019,Y2020,Y2021
0,1,Armenia,511,Total Population - Both sexes,1000 persons,NaN,NaN,NaN,NaN,NaN,...,2884.234,2897.588,2912.409,2925.553,2936.143,2944.791,2951.745,2957.731,2963.243,NaN
1,1,Armenia,645,Food supply quantity (kg/capita/yr),kg,NaN,NaN,NaN,NaN,NaN,...,2008.44,2028.09,2124.86,2158.98,2036.04,2004.51,1765.29,1760.49,NaN,NaN
2,1,Armenia,664,Food supply (kcal/capita/day),kcal/capita/day,NaN,NaN,NaN,NaN,NaN,...,2972.0,3020.0,3069.0,3090.0,3051.0,3072.0,3074.0,3022.0,NaN,NaN
3,1,Armenia,5123,Losses,1000 tonnes,NaN,NaN,NaN,NaN,NaN,...,421.0,465.0,576.0,634.0,532.0,445.0,435.0,375.0,NaN,NaN
4,1,Armenia,6111,Share of Total Expenditure,%,NaN,NaN,NaN,NaN,NaN,...,2.18,1.59,1.66,2.43,3.35,2.52,1.83,1.96,1.67,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2700,299,Palestine,210041,Prevalence of undernourishment (percent) (3-ye...,%,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2701,299,Palestine,210071,Number of severely food insecure people (milli...,millions,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.2,0.2,0.2,NaN,NaN
2702,299,Palestine,210081,Number of moderately or severely food insecure...,millions,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.3,1.3,1.3,NaN,NaN
2703,299,Palestine,210091,Prevalence of moderate or severe food insecuri...,%,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,26.3,26.3,26.3,NaN,NaN


In [48]:
mega_dataset = mega_dataset.append(purchasing_power_data)
mega_dataset

,Area Code,Area,Element Code,Element,Unit,Y1950,Y1951,Y1952,Y1953,Y1954,...,Y2012,Y2013,Y2014,Y2015,Y2016,Y2017,Y2018,Y2019,Y2020,Y2021
0,1,Armenia,511,Total Population - Both sexes,1000 persons,NaN,NaN,NaN,NaN,NaN,...,2884.234,2897.588,2912.409,2925.553,2936.143,2944.791,2951.745,2957.731,2963.243,NaN
1,1,Armenia,645,Food supply quantity (kg/capita/yr),kg,NaN,NaN,NaN,NaN,NaN,...,2008.44,2028.09,2124.86,2158.98,2036.04,2004.51,1765.29,1760.49,NaN,NaN
2,1,Armenia,664,Food supply (kcal/capita/day),kcal/capita/day,NaN,NaN,NaN,NaN,NaN,...,2972.0,3020.0,3069.0,3090.0,3051.0,3072.0,3074.0,3022.0,NaN,NaN
3,1,Armenia,5123,Losses,1000 tonnes,NaN,NaN,NaN,NaN,NaN,...,421.0,465.0,576.0,634.0,532.0,445.0,435.0,375.0,NaN,NaN
4,1,Armenia,6111,Share of Total Expenditure,%,NaN,NaN,NaN,NaN,NaN,...,2.18,1.59,1.66,2.43,3.35,2.52,1.83,1.96,1.67,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260,244,Samoa,NY.GDP.PCAP.PP.CD,"GDP per capita, PPP (current international $)",$,NaN,NaN,NaN,NaN,NaN,...,5478.932662,5508.715546,5569.505122,5825.05864,6330.806285,6489.776786,6537.875205,6912.03646,6766.228447,NaN
262,249,Yemen,NY.GDP.PCAP.PP.CD,"GDP per capita, PPP (current international $)",$,NaN,NaN,NaN,NaN,NaN,...,3491.20461,3688.519409,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
263,202,South Africa,NY.GDP.PCAP.PP.CD,"GDP per capita, PPP (current international $)",$,NaN,NaN,NaN,NaN,NaN,...,13215.708679,13606.942896,13602.102643,13701.94539,13748.461416,13860.270166,14209.093833,14289.760003,13355.598709,NaN
264,251,Zambia,NY.GDP.PCAP.PP.CD,"GDP per capita, PPP (current international $)",$,NaN,NaN,NaN,NaN,NaN,...,3422.619937,3578.847055,3539.43639,3430.400287,3404.67399,3485.02178,3606.097632,3617.200014,3456.322534,NaN


# Transposing

In [49]:
mega_dataset

,Area Code,Area,Element Code,Element,Unit,Y1950,Y1951,Y1952,Y1953,Y1954,...,Y2012,Y2013,Y2014,Y2015,Y2016,Y2017,Y2018,Y2019,Y2020,Y2021
0,1,Armenia,511,Total Population - Both sexes,1000 persons,NaN,NaN,NaN,NaN,NaN,...,2884.234,2897.588,2912.409,2925.553,2936.143,2944.791,2951.745,2957.731,2963.243,NaN
1,1,Armenia,645,Food supply quantity (kg/capita/yr),kg,NaN,NaN,NaN,NaN,NaN,...,2008.44,2028.09,2124.86,2158.98,2036.04,2004.51,1765.29,1760.49,NaN,NaN
2,1,Armenia,664,Food supply (kcal/capita/day),kcal/capita/day,NaN,NaN,NaN,NaN,NaN,...,2972.0,3020.0,3069.0,3090.0,3051.0,3072.0,3074.0,3022.0,NaN,NaN
3,1,Armenia,5123,Losses,1000 tonnes,NaN,NaN,NaN,NaN,NaN,...,421.0,465.0,576.0,634.0,532.0,445.0,435.0,375.0,NaN,NaN
4,1,Armenia,6111,Share of Total Expenditure,%,NaN,NaN,NaN,NaN,NaN,...,2.18,1.59,1.66,2.43,3.35,2.52,1.83,1.96,1.67,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260,244,Samoa,NY.GDP.PCAP.PP.CD,"GDP per capita, PPP (current international $)",$,NaN,NaN,NaN,NaN,NaN,...,5478.932662,5508.715546,5569.505122,5825.05864,6330.806285,6489.776786,6537.875205,6912.03646,6766.228447,NaN
262,249,Yemen,NY.GDP.PCAP.PP.CD,"GDP per capita, PPP (current international $)",$,NaN,NaN,NaN,NaN,NaN,...,3491.20461,3688.519409,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
263,202,South Africa,NY.GDP.PCAP.PP.CD,"GDP per capita, PPP (current international $)",$,NaN,NaN,NaN,NaN,NaN,...,13215.708679,13606.942896,13602.102643,13701.94539,13748.461416,13860.270166,14209.093833,14289.760003,13355.598709,NaN
264,251,Zambia,NY.GDP.PCAP.PP.CD,"GDP per capita, PPP (current international $)",$,NaN,NaN,NaN,NaN,NaN,...,3422.619937,3578.847055,3539.43639,3430.400287,3404.67399,3485.02178,3606.097632,3617.200014,3456.322534,NaN


In [50]:
features_dataset = mega_dataset[['Element Code', 'Element', 'Unit']]
features_dataset = features_dataset.drop_duplicates().reset_index(drop=True)

In [51]:
features_dataset

,Element Code,Element,Unit
0,511,Total Population - Both sexes,1000 persons
1,645,Food supply quantity (kg/capita/yr),kg
2,664,Food supply (kcal/capita/day),kcal/capita/day
3,5123,Losses,1000 tonnes
4,6111,Share of Total Expenditure,%
5,7271,Temperature change,°C
6,21032,Political stability and absence of violence/te...,index
7,23014,Food price inflation,%
8,210011,Number of people undernourished (million) (3-y...,millions
9,210041,Prevalence of undernourishment (percent) (3-ye...,%


In [52]:
years_columns = pd.DataFrame([x for x in mega_dataset.columns if x.startswith('Y19') or x.startswith('Y20')], columns=['Year'])
final_columns = [x for x in mega_dataset.columns
                 if not x.startswith('Y')
                 and x not in(['Element Code', 'Element', 'Unit'])] + ['Year'] + list(features_dataset['Element Code'])
final_dataset = pd.DataFrame(columns=final_columns)
final_areas = pd.DataFrame(mega_dataset['Area Code'].unique(), columns=['Area Code'])
final_areas = pd.merge(left=final_areas, right=area_code_map, how="left")

In [53]:
for area in final_areas['Area Code']:
    area_dataset = mega_dataset[
        mega_dataset['Area Code'] == area
    ][['Element Code'] + list(years_columns.Year)].set_index('Element Code').transpose().reset_index()
    area_dataset = area_dataset.rename(columns={'index': 'Year'})
    area_dataset['Area Code'] = area
    area_dataset['Area'] = final_areas[final_areas['Area Code'] == area].Area.values[0]
    final_dataset = final_dataset.append(area_dataset)
    del area_dataset
final_dataset = final_dataset.reset_index(drop=True)

In [54]:
final_dataset = final_dataset.rename(columns=dict(features_dataset[['Element Code', 'Element']].values))


In [55]:
final_dataset['Year'] = final_dataset.Year.str.replace('Y19', '19')
final_dataset['Year'] = final_dataset.Year.str.replace('Y20', '20')
final_dataset['Year'] = final_dataset.Year.astype(int)

In [58]:
final_dataset.to_csv('../data/final_dataset.csv')

In [59]:
features_dataset.to_csv('../data/features_description.csv')